## Population divergence

In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
def limitToRange(val, min_limit, max_limit):
    if val < min_limit: return min_limit
    if val > max_limit: return max_limit
    return val

def limitArrayValsToRange(vals, min_limit, max_limit):
    return [limitToRange(val, min_limit, max_limit) for val in vals]

In [3]:
def normalize_probability(arr):
    sum_ = sum(arr)
    return [a/sum_ for a in arr]

In [4]:
def populationDivergence(population_one, population_two):

    pop1_col_product = normalize_probability(np.prod(np.asarray(population_one), axis=0))
    pop2_col_product = normalize_probability(np.prod(np.asarray(population_two), axis=0))
    # print(population_one_colwise_product, population_two_colwise_product)
    
    # Kuhlback leibler divergence
    return np.mean([np.abs(np.log(pop1_col_product[i]/(pop2_col_product[i]+1e-64))) for i in range(len(pop1_col_product))])

### Experiment 1
All genes are biased in one direction.

In [5]:
def createIndividualExp1(size=10, bias=0.0, min_limit=1e-4, max_limit=1.0):
    return [limitToRange((random.randint(0,100) / 100) + bias, min_limit, max_limit) for i in range(size)]

In [6]:
def createPopulationExp1(size=10, bias=0.0):
    return [createIndividualExp1(10, bias=bias, min_limit=1e-4, max_limit=1.0) for i in range(size)]

In [7]:
def createPopulationsExp1(n_pops, n_individuals, bias_range=[-0.4,0.4]):
    pops = []
    biases = []
    for i in range(n_pops):
        bias = random.uniform(bias_range[0], bias_range[1])
        biases.append(bias)
        pops.append(createPopulationExp1(n_individuals, bias=bias))
    return pops, biases

In [8]:
# createPopulations(2, 2)

In [9]:
def testDivergenceMeasureExp1(n_pops, n_individuals, bias_range=[-0.4,0.4]):
    populations, biases = createPopulationsExp1(n_pops, n_individuals, bias_range)
    
    results = pd.DataFrame(columns=["population_1", "population_2","bias_1", "bias_2", "divergence"])
    
    #count = 0
    for i in range(n_pops):
        for j in range(n_pops):
            if j<i: 
                continue
                
            divergence = populationDivergence(populations[i], populations[j])

            results = results.append({"population_1":i, "population_2":j, 
                                      "bias_1": biases[i], "bias_2": biases[j], 
                                      "divergence":divergence}, ignore_index=True)
    
    return results

In [10]:
divergence_dataframe_exp1 = testDivergenceMeasureExp1(500, 100)

In [11]:
divergence_dataframe_exp1.to_csv("population_divergence_sample_experiment_1.csv",sep=",")

In [12]:
divergence_dataframe_exp1.head(10)

,population_1,population_2,bias_1,bias_2,divergence
0,0.0,0.0,0.196393,0.196393,0.000000
1,0.0,1.0,0.196393,0.124775,6.773405
2,0.0,2.0,0.196393,0.095541,7.454881
3,0.0,3.0,0.196393,-0.180980,38.995808
4,0.0,4.0,0.196393,0.044060,8.160659
5,0.0,5.0,0.196393,0.334821,4.574858
6,0.0,6.0,0.196393,0.386151,5.763887
7,0.0,7.0,0.196393,-0.014628,13.387120
8,0.0,8.0,0.196393,0.361635,6.655907
9,0.0,9.0,0.196393,-0.213342,19.989252


In [13]:
def createOneGenomeProduct():
    populations, biases = createPopulationsExp1(1, 100, bias_range=[-0.4,0.4])
    return np.prod(np.asarray(populations[0]), axis=0)
    

In [14]:
prod = createOneGenomeProduct()
prod

array([2.90628148e-43, 1.92144496e-49, 1.56218180e-44, 3.16069304e-45,
       1.75082127e-44, 1.06490734e-36, 4.21758147e-43, 7.22121104e-44,
       2.56186210e-43, 5.55230103e-49])

In [15]:
normalize_probability(prod)

[2.729137485975465e-07,
 1.8043288274121057e-13,
 1.4669635149247135e-08,
 2.9680421219058084e-09,
 1.6441050179786767e-08,
 0.9999989885739757,
 3.960510971871911e-07,
 6.781062970366795e-08,
 2.405711193639031e-07,
 5.213876555110231e-13]

### Experiment 2
Half of the genes in one direction, half in the other.  
Compare pairs where the directions are inverted,   

e.g.   
bias for pop 1 = [-0.3, -0,3, 0.3, 0.3]   
bias for pop 2 = [0.3, 0.3, -0.3, -0.3]

In [16]:
def createIndividualExp2(size=10, biases=[0.0,0.0], min_limit=1e-4, max_limit=1.0):
    return [limitToRange((random.randint(0,100) / 100) + (biases[0] if i < size//2 else biases[1]), min_limit, max_limit) for i in range(size)]

In [17]:
def createPopulationExp2(size=10, biases=[0.0,0.0]):
    return [createIndividualExp2(10, biases=biases, min_limit=1e-4, max_limit=1.0) for i in range(size)]

In [18]:
def createPopulationsExp2(n_pairs, n_individuals, bias_range=[-0.4,0.4]):
    pops = []
    biases = []
    pairs = []
    for i in range(n_pairs):
        bias = random.uniform(bias_range[0], bias_range[1])
        
        # Create pair of populations with inverted bias directions 
        pops.append(createPopulationExp2(n_individuals, biases=[bias, -bias]))
        pops.append(createPopulationExp2(n_individuals, biases=[-bias, bias]))
        
        biases.append(bias)
        pairs.append(i)
    return pops, biases, pairs

In [19]:
def testDivergenceMeasureExp2(n_pairs, n_individuals, bias_range=[-0.4,0.4]):
    populations, biases, pairs = createPopulationsExp2(n_pairs, n_individuals, bias_range)
    
    results = pd.DataFrame(columns=["population_1", "population_2",
                                    "bias_1_first", "bias_1_second", 
                                    "bias_2_first","bias_2_second", 
                                    "pair", "divergence"])
    
    for i in range(n_pairs):
        ind_pop_1 = i*2
        ind_pop_2 = i*2 + 1
                
        divergence = populationDivergence(populations[ind_pop_1], populations[ind_pop_2])

        results = results.append({"population_1":ind_pop_1, "population_2":ind_pop_2, 
                          "bias_1_first": biases[i], "bias_1_second": -biases[i],
                          "bias_2_first": - biases[i], "bias_2_second": biases[i],
                          "pair": pairs[i] ,"divergence":divergence}, ignore_index=True)
    
    return results

In [20]:
divergence_dataframe_exp2 = testDivergenceMeasureExp2(500, 100)

In [21]:
divergence_dataframe_exp2.to_csv("population_divergence_sample_experiment_2.csv",sep=",")

In [22]:
divergence_dataframe_exp2.head(10)

,population_1,population_2,bias_1_first,bias_1_second,bias_2_first,bias_2_second,pair,divergence
0,0.0,1.0,-0.054477,0.054477,0.054477,-0.054477,0.0,72.054298
1,2.0,3.0,0.170389,-0.170389,-0.170389,0.170389,1.0,173.385569
2,4.0,5.0,0.095977,-0.095977,-0.095977,0.095977,2.0,110.923134
3,6.0,7.0,0.113135,-0.113135,-0.113135,0.113135,3.0,128.963617
4,8.0,9.0,-0.171245,0.171245,0.171245,-0.171245,4.0,163.747962
5,10.0,11.0,0.136270,-0.136270,-0.136270,0.136270,5.0,152.237938
6,12.0,13.0,-0.245981,0.245981,0.245981,-0.245981,6.0,202.669933
7,14.0,15.0,-0.130766,0.130766,0.130766,-0.130766,7.0,159.499099
8,16.0,17.0,-0.002165,0.002165,0.002165,-0.002165,8.0,10.943944
9,18.0,19.0,-0.149962,0.149962,0.149962,-0.149962,9.0,148.013766
